In [1]:
import os
import torch
import torch.backends.mps as mps
from torchvision import datasets, transforms


In [2]:
if mps.is_available():
    use_mps = True
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
        ])


In [4]:
download_state = True if not os.path.exists('./data/MNIST') else False
trainset = datasets.MNIST(root='./data', train=True, download=download_state, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
test_set = datasets.MNIST(root='./data', train=False, download=download_state, transform=transform)
testloader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)

In [5]:
from torch import nn
from torch.nn import functional as F
class CNN_model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1 = nn.Conv2d(1, 32, 3,1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3,1)
        self.fc1 = nn.Linear(16 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        output = F.softmax(self.fc3(x), dim=1)
        return output

model = CNN_model().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
train_loss = []
train_count = []
test_loss = []
test_count = []

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
            train_loss.append(loss.item())
            train_count.append((batch_idx*4) + ((epoch-1)*len(trainloader.dataset)))
    
def test(epoch):
    model.eval()
    test_losses = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_losses += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_losses /= len(testloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))
    test_loss.append(test_loss)
    test_count.append((epoch-1)*len(trainloader.dataset))
    

In [7]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

: 